<a href="https://colab.research.google.com/github/41371115h-dot/114-class-thursday2-4/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil

In [17]:
!pip install folium

In [18]:
import os, random
import pandas as pd
import folium
import gradio as gr
from google.colab import auth
import gspread
from google.auth import default
import google.generativeai as genai
from IPython.display import HTML

In [19]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [20]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1XxIVzJXzHps5Ylhvdv1PAHYRDpmKdkYu7e4Leblj63M/edit?usp=sharing"
WORKSHEET_NAME = "餐廳列表"
CENTER_LAT, CENTER_LNG = 24.975279207409475, 121.54292109362143

In [21]:
# --- 載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])

In [22]:
# 清理欄位名稱（全形／半形統一）
df_restaurants.columns = (
    df_restaurants.columns
    .str.strip()
    .str.replace('／', '/')
    .str.replace('（', '(')
    .str.replace('）', ')')
)

In [23]:
import os
import google.generativeai as genai

if "ella4" not in os.environ:
    print("⚠️ 找不到 Secret：ella4，請先在 Colab Secrets 加入 API key 或手動設定。")
    os.environ["ella4"] = input("請貼上你的 Gemini API Key：").strip()

genai.configure(api_key=os.environ["ella4"])
model = genai.GenerativeModel("gemini-1.5-flash")

In [24]:
# --- 主要功能函式 ---
def recommend_restaurants():
    # 隨機抽 3 家
    sample = df_restaurants.sample(3, random_state=random.randint(1, 9999))

    # 建立地圖
    m = folium.Map(location=[CENTER_LAT, CENTER_LNG], zoom_start=14)
    for _, row in sample.iterrows():
        folium.Marker(
            location=[float(row['lat']), float(row['lng'])],
            tooltip=row['店家名稱'],
            popup=f"{row['類別/招牌品項']}<br>{row['備註/特色']}",
            icon=folium.Icon(icon="cloud")
        ).add_to(m)

    # 生成 AI 分析
    prompt = "這是三家位於新店區的餐廳資料，請用中文幫我以「優缺點＋快速決策建議」壓成100字內摘要：\n\n"
    for i, r in sample.iterrows():
        prompt += f"{r['店家名稱']}：{r['類別/招牌品項']}，特色：{r['備註/特色']}\n"

    response = model.generate_content(prompt)
    ai_summary = response.text.strip()

    # 回傳地圖 HTML、表格、AI 建議
    map_html = m._repr_html_()
    return map_html, sample[['店家名稱', '類別/招牌品項', '參考地點(捷運站/商圈)', '備註/特色']], ai_summary

In [25]:
def recommend_random_restaurants():
    # 隨機取三間
    sample_df = df_restaurants.sample(3).reset_index(drop=True)

    # 組合給 AI 的文字
    info_text = "\n".join([
        f"{i+1}. {row['店家名稱']}：{row['類別/招牌品項']}，地點 {row['參考地點 (捷運站/商圈)']}，特色：{row['備註/特色']}"
        for i, row in sample_df.iterrows()
    ])

    prompt = f"""
    以下是三家位於新北市新店區的餐廳：
    {info_text}

    請以專業但親切的語氣，幫我在 100 字內分析這三家的優缺點，
    並給出簡短建議，幫助使用者快速決策要去哪一家。
    """

    ai_reply = model.generate_content(prompt).text

    return sample_df, ai_reply

In [26]:
# ==========================
# Gradio 介面
# ==========================
with gr.Blocks(title="新店美食 AI 推薦") as demo:
    gr.Markdown("## 🍜 新北市新店區美食 AI 推薦系統")
    gr.Markdown("點擊下方按鈕，系統將隨機推薦 3 家餐廳，並由 AI 幫你壓成 100 字建議文案。")

    recommend_btn = gr.Button("🎲 隨機推薦 3 家")
    table_output = gr.DataFrame(label="推薦餐廳清單")
    ai_output = gr.Textbox(label="AI 建議（100 字）", lines=5, max_lines=10, interactive=False)

    recommend_btn.click(
        fn=recommend_random_restaurants,
        inputs=[],
        outputs=[table_output, ai_output]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://568e3ad9daccfb319a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
